https://jira.x5.ru/browse/CVMXC-2717

In [1]:
name = 'CVMXC-2717_j7'

In [2]:
import sys
import pandas as pd
import pyspark.sql.functions as F
sys.path.append('/home/jovyan/glow-byte-filters-pyspark')
sys.path.append('/home/jovyan/x5_some_tasks/Osipov/')
import datetime
from datetime import timedelta
from spark import *
from utils_osipov import *
from logic_filters import *
from segmentation import *

%load_ext autoreload
%autoreload 2

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 500

In [3]:
sc, spark = restart_spark(name, 100, executor_memory="3G", executor_cores=5, driver_memory="12G",
                          additional_params={"spark.sql.shuffle.partitions": "200"})
sc.setLogLevel('ERROR')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/17 00:54:43 WARN cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to request executors before the AM has registered!


In [4]:
LOYALTY_CARDS = "hive_ssa_tc5.loyalty_card"
LOYALTY_CARDHOLDERS = "hive_ssa_tc5.loyalty_cardholder"
ACCOUNTS = "hive_ssa_tc5.account"
CVM5_GUESTS = "hive_cvm_acrm.cvm5_guests"

DIM_STORE = "hive_ssa_main.dim_store"
CHECKS_HEADERS = "hive_ssa_main.fct_rtl_txn"
CHECKS_ITEMS = "hive_ssa_main.fct_rtl_txn_item"
PRODUCTS = "hive_ssa_tc5.cvm_product"

### Выбираем гостей нужного юзкейса

In [39]:
usecase = ['upgrade', 'cross', 'ump']
dt = datetime.date(2022, 2, 16)
lifetime = 90
freq = 1

In [6]:
customers_usecase = (spark
                     .table(CVM5_GUESTS)
                     .filter(F.to_date('calculation_dt') == dt)
                     .filter(F.col('usecase').isin(usecase))
                     .filter(F.col('lifetime') >= lifetime)
                     .filter(F.col('frequency') >= freq)
                     .select('account_no', 'customer_rk')
                            )


In [7]:
print(customers_usecase.count()) #16775430

[Stage 0:========================================>                 (7 + 3) / 10]

15869655


### 1 проверяем на доступность отобранных гостей в определенную дату

In [8]:
check_date = '2022-02-28'

In [9]:
seg_sms1 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[0], 
                                         check_date=check_date, 
                                         debug_mode=False
                                        )
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:09


In [10]:
seg_sms2 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[1], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:06


In [11]:
seg_sms3 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[2], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:06


In [12]:
seg_sms = seg_sms1.union(seg_sms2).union(seg_sms3).distinct()

In [5]:
# seg_sms.write.parquet('temp2717_1', mode='overwrite')
seg_sms = spark.read.parquet('temp2717_1')

In [16]:
seg_sms.count() #

4921531

In [17]:
slip1 = (slip_check_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[0], 
                                         check_date=check_date, 
                                         debug_mode=False
                                        )
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:03


In [18]:
slip2 = (slip_check_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[1], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:02


In [19]:
slip3 = (slip_check_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[2], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:02


In [20]:
slip = slip1.union(slip2).union(slip3).distinct()

In [6]:
# slip.write.parquet('temp2717_2', mode='overwrite')
slip = spark.read.parquet('temp2717_2')

In [22]:
slip.count() #

7272613

### Проверка на доступность плю в магазинах

РФ

In [7]:
plu_codes = [3977668,3977669,3977670,4113609]

In [8]:
pplu = spark.createDataFrame(pd.DataFrame({"plu_code": plu_codes}))

shops = (spark.table(DIM_STORE)
            .filter(F.col('valid_to_dttm')== datetime.datetime(5999, 1, 1, 0, 0))
#             .filter(F.col('federal_subject_dk').isin(federal_subject_dk))
            .selectExpr('store_id as plant')
            .distinct()
            )


tc5_stores_assort = (
    spark.table('HIVE_SSA_MAIN.ASSORTMENT_X_PLU_X_STORE')
    .withColumnRenamed('plu_id', 'plu_code')
    .withColumnRenamed('store_id', 'plant')
    .filter(F.col("plu_negate_flg") != 1)
    .selectExpr('plu_code', 'plant')
    .join(F.broadcast(pplu), 'plu_code', 'inner')
)

plants = (shops.join(tc5_stores_assort, on='plant', how='inner')
                .select('plant')
                .distinct()
                .toPandas()['plant']
                .tolist()
         )

len(plants)

20978

### 1

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [25]:
start_date = datetime.date(2021, 11, 15)
end_date = datetime.date(2022, 2, 14)

In [26]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [27]:
checks_headers_tc5 = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

22/02/16 19:01:17 ERROR client.TransportResponseHandler: Still have 1 requests outstanding when connection from /192.168.234.58:55608 is closed
22/02/16 19:01:17 ERROR client.TransportClient: Failed to send RPC RPC 9048932819183197655 to /192.168.234.58:55608: java.nio.channels.ClosedChannelException
java.nio.channels.ClosedChannelException
	at io.netty.channel.AbstractChannel$AbstractUnsafe.write(...)(Unknown Source)
22/02/16 19:01:17 ERROR client.TransportResponseHandler: Still have 2 requests outstanding when connection from /192.168.234.58:55610 is closed
22/02/16 19:01:18 ERROR client.TransportResponseHandler: Still have 2 requests outstanding when connection from /192.168.234.58:55612 is closed
22/02/16 19:01:18 ERROR client.TransportResponseHandler: Still have 2 requests outstanding when connection from /192.168.234.58:55616 is closed
22/02/16 19:01:18 ERROR client.TransportResponseHandler: Still have 2 requests outstanding when connection from /192.168.234.58:55614 is closed
22

In [28]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(seg_sms, on='customer_rk', how='inner')

In [29]:
checks_headers_tc5 = checks_headers_tc5.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [30]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [31]:
# plu_hierarchy_lvl_4_dk = ['FR0604001', 'FR0608005', 'FR0604002', 'FR0608001'] #сметана, молоко, кефир
syntethic_category_id = [5,66,92,108]
# plu_brand_code = ['2452']
# plu_vendor_nm = ['Данон']
# plu_not_in = [4138521]

In [32]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
#                  .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
#                  .filter(F.col('plu_vendor_nm').isin(plu_vendor_nm))
                 .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
#                  .filter(F.col('plu_brand_code').isin(plu_brand_code))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [33]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [34]:
checks_tc5 = checks_items.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [35]:
accs = ['account_no']
pdf = checks_tc5.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf = pdf.filter(F.col('base_qty_per_period') >= 1)
checks_tc5 = checks_tc5.join(pdf, on=accs)

In [36]:
seg1 = (checks_tc5.select('customer_rk')
                  .distinct()
      )

In [37]:
checks_headers2 = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [38]:
checks_headers_tc52 = checks_headers2.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [39]:
loyalty_cards2 = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders2 = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info2 = loyalty_cards2.join(loyalty_cardholders2, on='account_no', how='inner')
clients_info2 = clients_info2.join(seg_sms, on='customer_rk', how='inner')

In [40]:
checks_headers_tc52 = checks_headers_tc52.join(clients_info2, on='loyalty_card_no') #оставили чеки только нужных гостей

In [41]:
checks_items2 = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [42]:
plu_codes_cat = [3190682, 3245437, 3196547, 3404062, 3965621, 3190706, 3941462, 
                 3191950, 3612089, 3612131, 3422623, 3476943]

In [43]:
checks_items2 = checks_items2.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [44]:
checks_tc52 = checks_items2.join(checks_headers_tc52, 'rtl_txn_id', how='inner')

In [45]:
seg2 = (checks_tc52.select('customer_rk')
                  .distinct()
      )

In [46]:
seg_1 = seg1.join(seg2, 'customer_rk', 'left_anti')

In [47]:
seg_1 = seg_1.withColumn('label', F.lit('sms'))

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [52]:
start_date = datetime.date(2021, 11, 15)
end_date = datetime.date(2022, 2, 14)

In [53]:
checks_headers3 = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [54]:
checks_headers_tc53 = checks_headers3.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [55]:
loyalty_cards3 = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders3 = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info3 = loyalty_cards3.join(loyalty_cardholders3, on='account_no', how='inner')
clients_info3 = clients_info3.join(slip, on='customer_rk', how='inner')

In [56]:
checks_headers_tc53 = checks_headers_tc53.join(clients_info3, on='loyalty_card_no') #оставили чеки только нужных гостей

In [57]:
checks_items3 = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [58]:
# plu_hierarchy_lvl_4_dk = ['FR0604001', 'FR0608005', 'FR0604002', 'FR0608001'] #сметана, молоко, кефир
syntethic_category_id = [5,66,92,108]
# plu_brand_code = ['2452']
# plu_vendor_nm = ['Данон']
# plu_not_in = [4138521]

In [59]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
#                  .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
#                  .filter(F.col('plu_vendor_nm').isin(plu_vendor_nm))
                 .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
#                  .filter(F.col('plu_brand_code').isin(plu_brand_code))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [60]:
checks_items3 = checks_items3.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [61]:
checks_tc53 = checks_items3.join(checks_headers_tc53, 'rtl_txn_id', how='inner')

In [62]:
accs = ['account_no']
pdf3 = checks_tc53.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf3 = pdf3.filter(F.col('base_qty_per_period') >= 1)
checks_tc53 = checks_tc53.join(pdf3, on=accs)

In [63]:
seg3 = (checks_tc53.select('customer_rk')
                  .distinct()
      )

In [64]:
checks_headers4 = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [65]:
checks_headers_tc54 = checks_headers4.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [66]:
loyalty_cards4 = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders4 = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info4 = loyalty_cards4.join(loyalty_cardholders4, on='account_no', how='inner')
clients_info4 = clients_info4.join(slip, on='customer_rk', how='inner')

In [67]:
checks_headers_tc54 = checks_headers_tc54.join(clients_info4, on='loyalty_card_no') #оставили чеки только нужных гостей

In [68]:
checks_items4 = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [69]:
plu_codes_cat = [3190682, 3245437, 3196547, 3404062, 3965621, 3190706, 3941462, 
                 3191950, 3612089, 3612131, 3422623, 3476943]

In [70]:
checks_items4 = checks_items4.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [71]:
checks_tc54 = checks_items4.join(checks_headers_tc54, 'rtl_txn_id', how='inner')

In [72]:
seg4 = (checks_tc54.select('customer_rk')
                  .distinct()
      )

In [73]:
seg_2 = seg3.join(seg4, 'customer_rk', 'left_anti')

In [74]:
seg_2 = seg_2.withColumn('label', F.lit('slip'))

In [75]:
seg = seg_1.union(seg_2).distinct()

In [76]:
seg.write.parquet('temp2717_seg1', mode='overwrite')
seg = spark.read.parquet('temp2717_seg1')

In [77]:
seg_pd1 = seg.toPandas()

In [78]:
seg_pd1.head()

,customer_rk,label
0,11921657,sms
1,12421689,sms
2,13105802,sms
3,13204800,sms
4,13771741,sms


In [79]:
seg_no = 'seg1'

In [80]:
seg_pd1.to_csv(name + '_' + seg_no + '_All_uc' + str(dt) + '.csv', index=False)

In [41]:
seg_pd1 = pd.read_csv('CVMXC-2717_j7_seg1_All_uc2022-02-16.csv')

In [46]:
seg_pd1['customer_rk'] = seg_pd1['customer_rk'].astype('str')

In [47]:
seg_pd1_sms = seg_pd1[seg_pd1.label == 'sms']
seg_pd1_slip = seg_pd1[seg_pd1.label == 'slip']

In [48]:
cross, upgrade, ump = get_usecases(seg_pd1_sms)

In [49]:
print('cross: {}, upgrade: {}, ump: {} (all: {})'.format(len(cross), len(upgrade), len(ump), len(cross)+len(upgrade)+len(ump)))

cross: 1292035, upgrade: 1303795, ump: 1175162 (all: 3770992)


In [50]:
cross, upgrade, ump = get_usecases(seg_pd1_slip)

In [51]:
print('cross: {}, upgrade: {}, ump: {} (all: {})'.format(len(cross), len(upgrade), len(ump), len(cross)+len(upgrade)+len(ump)))

cross: 1861405, upgrade: 1610916, ump: 1718153 (all: 5190474)


### 2

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [84]:
start_date = datetime.date(2021, 11, 15)
end_date = datetime.date(2022, 2, 14)

In [85]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [86]:
checks_headers_tc5 = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [87]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(seg_sms, on='customer_rk', how='inner')

In [88]:
checks_headers_tc5 = checks_headers_tc5.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [89]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [90]:
plu_hierarchy_lvl_4_dk = ['FR0609003'] #сметана, молоко, кефир
syntethic_category_id = [26, 51]
# plu_brand_code = ['2452']
# plu_vendor_nm = ['Данон']
# plu_not_in = [4138521]

In [91]:
plu_codes_cat1 = (spark
                 .table(PRODUCTS)
                 .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
                 .select('plu_id').distinct().toPandas()['plu_id'].tolist()
                )
plu_codes_cat2 = (spark
                 .table(PRODUCTS)
                 .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
                 .select('plu_id').distinct().toPandas()['plu_id'].tolist()
                )
plu_codes_cat = plu_codes_cat1 + plu_codes_cat2

In [92]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [93]:
checks_tc5 = checks_items.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [94]:
accs = ['account_no']
pdf = checks_tc5.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf = pdf.filter(F.col('base_qty_per_period') >= 1)
checks_tc5 = checks_tc5.join(pdf, on=accs)

In [95]:
seg1 = (checks_tc5.select('customer_rk')
                  .distinct()
      )

In [96]:
checks_headers2 = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [97]:
checks_headers_tc52 = checks_headers2.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [98]:
loyalty_cards2 = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders2 = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info2 = loyalty_cards2.join(loyalty_cardholders2, on='account_no', how='inner')
clients_info2 = clients_info2.join(seg_sms, on='customer_rk', how='inner')

In [99]:
checks_headers_tc52 = checks_headers_tc52.join(clients_info2, on='loyalty_card_no') #оставили чеки только нужных гостей

In [100]:
checks_items2 = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [101]:
plu_codes_cat = [3190682, 3245437, 3196547, 3404062, 3965621, 3190706, 3941462, 
                 3191950, 3612089, 3612131, 3422623, 3476943]

In [102]:
checks_items2 = checks_items2.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [103]:
checks_tc52 = checks_items2.join(checks_headers_tc52, 'rtl_txn_id', how='inner')

In [104]:
seg2 = (checks_tc52.select('customer_rk')
                  .distinct()
      )

In [105]:
seg_1 = seg1.join(seg2, 'customer_rk', 'left_anti')

In [106]:
seg_1 = seg_1.withColumn('label', F.lit('sms'))

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [107]:
start_date = datetime.date(2021, 11, 15)
end_date = datetime.date(2022, 2, 14)

In [108]:
checks_headers3 = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [109]:
checks_headers_tc53 = checks_headers3.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [110]:
loyalty_cards3 = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders3 = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info3 = loyalty_cards3.join(loyalty_cardholders3, on='account_no', how='inner')
clients_info3 = clients_info3.join(slip, on='customer_rk', how='inner')

In [111]:
checks_headers_tc53 = checks_headers_tc53.join(clients_info3, on='loyalty_card_no') #оставили чеки только нужных гостей

In [112]:
checks_items3 = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [113]:
plu_hierarchy_lvl_4_dk = ['FR0609003'] #сметана, молоко, кефир
syntethic_category_id = [26, 51]
# plu_brand_code = ['2452']
# plu_vendor_nm = ['Данон']
# plu_not_in = [4138521]

In [114]:
plu_codes_cat1 = (spark
                 .table(PRODUCTS)
                 .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
                 .select('plu_id').distinct().toPandas()['plu_id'].tolist()
                )
plu_codes_cat2 = (spark
                 .table(PRODUCTS)
                 .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
                 .select('plu_id').distinct().toPandas()['plu_id'].tolist()
                )
plu_codes_cat = plu_codes_cat1 + plu_codes_cat2

In [115]:
checks_items3 = checks_items3.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [116]:
checks_tc53 = checks_items3.join(checks_headers_tc53, 'rtl_txn_id', how='inner')

In [117]:
accs = ['account_no']
pdf3 = checks_tc53.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf3 = pdf3.filter(F.col('base_qty_per_period') >= 1)
checks_tc53 = checks_tc53.join(pdf3, on=accs)

In [118]:
seg3 = (checks_tc53.select('customer_rk')
                  .distinct()
      )

In [119]:
checks_headers4 = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [120]:
checks_headers_tc54 = checks_headers4.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [121]:
loyalty_cards4 = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders4 = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info4 = loyalty_cards4.join(loyalty_cardholders4, on='account_no', how='inner')
clients_info4 = clients_info4.join(slip, on='customer_rk', how='inner')

In [122]:
checks_headers_tc54 = checks_headers_tc54.join(clients_info4, on='loyalty_card_no') #оставили чеки только нужных гостей

In [123]:
checks_items4 = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [124]:
plu_codes_cat = [3190682, 3245437, 3196547, 3404062, 3965621, 3190706, 3941462, 
                 3191950, 3612089, 3612131, 3422623, 3476943]

In [125]:
checks_items4 = checks_items4.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [126]:
checks_tc54 = checks_items4.join(checks_headers_tc54, 'rtl_txn_id', how='inner')

In [127]:
seg4 = (checks_tc54.select('customer_rk')
                  .distinct()
      )

In [128]:
seg_2 = seg3.join(seg4, 'customer_rk', 'left_anti')

In [129]:
seg_2 = seg_2.withColumn('label', F.lit('slip'))

In [130]:
seg = seg_1.union(seg_2).distinct()

In [131]:
seg.write.parquet('temp2717_seg2', mode='overwrite')
seg = spark.read.parquet('temp2717_seg2')

[Stage 230:(207 + 3) / 210][Stage 233:(461 + 232) / 1345][Stage 236:(969 + 71) / 7013]]]22/02/16 20:06:08 ERROR cluster.YarnScheduler: Lost executor 587 on mn-hd0255.x5.ru: Unable to create executor due to Unable to register with external shuffle server due to : java.util.concurrent.TimeoutException: Timeout waiting for task.
[Stage 232:(185 + 15) / 200][Stage 233:(1341 + 4) / 1345][Stage 236:(1541 + 125) / 7013]]22/02/16 20:07:05 ERROR cluster.YarnScheduler: Lost executor 652 on mn-hd0206.x5.ru: Unable to create executor due to Unable to register with external shuffle server due to : java.util.concurrent.TimeoutException: Timeout waiting for task.


In [132]:
seg_pd2 = seg.toPandas()

In [133]:
seg_pd2.head()

,customer_rk,label
0,11413190,slip
1,12272117,slip
2,13135917,slip
3,13286271,slip
4,14065681,slip


In [134]:
seg_no = 'seg2'

In [135]:
seg_pd2.to_csv(name + '_' + seg_no + '_All_uc' + str(dt) + '.csv', index=False)

In [52]:
seg_pd2 = pd.read_csv('CVMXC-2717_j7_seg2_All_uc2022-02-16.csv')

In [53]:
seg_pd2['customer_rk'] = seg_pd2['customer_rk'].astype('str')

In [54]:
seg_pd2_sms = seg_pd2[seg_pd2.label == 'sms']
seg_pd2_slip = seg_pd2[seg_pd2.label == 'slip']

In [55]:
cross, upgrade, ump = get_usecases(seg_pd2_sms)

In [56]:
print('cross: {}, upgrade: {}, ump: {} (all: {})'.format(len(cross), len(upgrade), len(ump), len(cross)+len(upgrade)+len(ump)))

cross: 1062326, upgrade: 1068034, ump: 966177 (all: 3096537)


In [57]:
cross, upgrade, ump = get_usecases(seg_pd2_slip)

In [58]:
print('cross: {}, upgrade: {}, ump: {} (all: {})'.format(len(cross), len(upgrade), len(ump), len(cross)+len(upgrade)+len(ump)))

cross: 1627957, upgrade: 1407488, ump: 1503874 (all: 4539319)


### 3

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [9]:
start_date = datetime.date(2021, 11, 15)
end_date = datetime.date(2022, 2, 14)

In [10]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [11]:
checks_headers_tc5 = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [12]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(seg_sms, on='customer_rk', how='inner')

In [13]:
checks_headers_tc5 = checks_headers_tc5.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [14]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [15]:
plu_codes_cat = [3190693, 3196547, 3190698, 3404062, 3190682, 3612131, 3941462, 
                 3245437, 3965621, 3190706, 3612089, 3191950]

In [16]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [17]:
checks_tc5 = checks_items.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [18]:
accs = ['account_no']
pdf = checks_tc5.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf = pdf.filter(F.col('base_qty_per_period').isin([i for i in range(1, 13)]))
checks_tc5 = checks_tc5.join(pdf, on=accs)

In [19]:
seg_1 = (checks_tc5.select('customer_rk')
                  .distinct()
      )

In [20]:
seg_1 = seg_1.withColumn('label', F.lit('sms'))

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [21]:
start_date = datetime.date(2021, 11, 15)
end_date = datetime.date(2022, 2, 14)

In [22]:
checks_headers3 = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [23]:
checks_headers_tc53 = checks_headers3.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [24]:
loyalty_cards3 = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders3 = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info3 = loyalty_cards3.join(loyalty_cardholders3, on='account_no', how='inner')
clients_info3 = clients_info3.join(slip, on='customer_rk', how='inner')

In [25]:
checks_headers_tc53 = checks_headers_tc53.join(clients_info3, on='loyalty_card_no') #оставили чеки только нужных гостей

In [26]:
checks_items3 = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [27]:
plu_codes_cat = [3190693, 3196547, 3190698, 3404062, 3190682, 3612131, 
                 3941462, 3245437, 3965621, 3190706, 3612089, 3191950]

In [28]:
checks_items3 = checks_items3.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [29]:
checks_tc53 = checks_items3.join(checks_headers_tc53, 'rtl_txn_id', how='inner')

In [30]:
accs = ['account_no']
pdf3 = checks_tc53.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf3 = pdf3.filter(F.col('base_qty_per_period') >= 1)
checks_tc53 = checks_tc53.join(pdf3, on=accs)

In [31]:
seg_2 = (checks_tc53.select('customer_rk')
                  .distinct()
      )

In [32]:
seg_2 = seg_2.withColumn('label', F.lit('slip'))

In [33]:
seg = seg_1.union(seg_2).distinct()

In [34]:
seg.write.parquet('temp2717_seg3', mode='overwrite')
seg = spark.read.parquet('temp2717_seg3')

In [35]:
seg_pd3 = seg.toPandas()

In [36]:
seg_pd3.head()

,customer_rk,label
0,25467222,sms
1,13662381,sms
2,58270662,sms
3,37901990,sms
4,52612789,sms


In [37]:
seg_no = 'seg3'

In [59]:
seg_pd3.to_csv(name + '_' + seg_no + '_All_uc' + str(dt) + '.csv', index=False)

In [60]:
seg_pd3['customer_rk'] = seg_pd3['customer_rk'].astype('str')

In [61]:
seg_pd3_sms = seg_pd3[seg_pd3.label == 'sms']
seg_pd3_slip = seg_pd3[seg_pd3.label == 'slip']

In [62]:
cross, upgrade, ump = get_usecases(seg_pd3_sms)

In [63]:
print('cross: {}, upgrade: {}, ump: {} (all: {})'.format(len(cross), len(upgrade), len(ump), len(cross)+len(upgrade)+len(ump)))

cross: 92268, upgrade: 98759, ump: 84889 (all: 275916)


In [64]:
cross, upgrade, ump = get_usecases(seg_pd3_slip)

In [65]:
print('cross: {}, upgrade: {}, ump: {} (all: {})'.format(len(cross), len(upgrade), len(ump), len(cross)+len(upgrade)+len(ump)))

cross: 149799, upgrade: 130626, ump: 140747 (all: 421172)
